# uv --link-mode=copy / --active 検証

## 目的

Databricks の `/Workspace` 上で `uv run` を動作させる方法を検証する。

## 検証項目

1. `--link-mode=copy`: パッケージのリンクモードをコピーに変更
2. `--active`: 既存の Databricks 環境を使用

## 1. 環境確認

In [ ]:
import os
import subprocess

cwd = os.getcwd()
print(f"CWD: {cwd}")
print(f"Python: {os.popen('which python').read().strip()}")
print(f"/Workspace 上か: {cwd.startswith('/Workspace')}")
print(f"VIRTUAL_ENV: {os.environ.get('VIRTUAL_ENV', 'Not set')}")

pyproject_exists = os.path.exists(os.path.join(cwd, 'pyproject.toml'))
print(f"pyproject.toml exists: {pyproject_exists}")

PROJECT_DIR = cwd if pyproject_exists else None

In [ ]:
# ファイルシステム情報
print("=== ファイルシステム情報 ===")
!df -h /Workspace 2>/dev/null || echo "/Workspace の df 取得失敗"
print()
!df -h /usr/bin 2>/dev/null || echo "/usr/bin の df 取得失敗"
print()
virtual_env = os.environ.get('VIRTUAL_ENV', '')
if virtual_env:
    print(f"=== VIRTUAL_ENV ({virtual_env}) ===")
    !df -h "$VIRTUAL_ENV" 2>/dev/null || echo "VIRTUAL_ENV の df 取得失敗"

## 2. uv インストール

In [ ]:
%pip install uv -q

In [ ]:
!uv --version

## 3. uv sync --active (Databricks 既存環境を使用)

In [ ]:
if PROJECT_DIR:
    result = subprocess.run(
        ["uv", "sync", "--no-dev", "--active"],
        cwd=PROJECT_DIR,
        capture_output=True,
        text=True
    )
    print("=== uv sync --no-dev --active ===")
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    print(f"Return code: {result.returncode}")
else:
    print("PROJECT_DIR が設定されていません")

In [ ]:
# uv run --active を試す
if PROJECT_DIR:
    result = subprocess.run(
        ["uv", "run", "--no-dev", "--active", "python", "-c", "import httpx; print(f'httpx version: {httpx.__version__}')"],
        cwd=PROJECT_DIR,
        capture_output=True,
        text=True
    )
    print("=== uv run --no-dev --active ===")
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    print(f"Return code: {result.returncode}")

## 4. uv sync (比較用: .venv 作成)

In [ ]:
import shutil

# .venv が存在すれば削除
venv_path = os.path.join(PROJECT_DIR, ".venv") if PROJECT_DIR else None
if venv_path and os.path.exists(venv_path):
    shutil.rmtree(venv_path)
    print(f"Removed {venv_path}")

In [ ]:
if PROJECT_DIR:
    result = subprocess.run(
        ["uv", "sync", "--no-dev"],
        cwd=PROJECT_DIR,
        capture_output=True,
        text=True
    )
    print("=== uv sync --no-dev ===")
    print(result.stderr)
    print(f"Return code: {result.returncode}")

In [ ]:
# .venv の構造を確認
print("=== .venv/bin/ の内容 ===")
!ls -la .venv/bin/ 2>/dev/null | head -20

print("\n=== python シンボリックリンクの確認 ===")
!ls -la .venv/bin/python 2>/dev/null
!file .venv/bin/python 2>/dev/null
!readlink .venv/bin/python 2>/dev/null || echo "readlink 失敗"
!readlink -f .venv/bin/python 2>/dev/null || echo "readlink -f 失敗"

In [ ]:
# uv run を試す (.venv 使用)
if PROJECT_DIR:
    result = subprocess.run(
        ["uv", "run", "--no-dev", "python", "-c", "import httpx; print(f'httpx version: {httpx.__version__}')"],
        cwd=PROJECT_DIR,
        capture_output=True,
        text=True
    )
    print("=== uv run (--active なし、.venv 使用) ===")
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    print(f"Return code: {result.returncode}")

## 5. 結論

検証結果をここに記載する